In [2]:
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
from dataclasses import dataclass
from datetime import datetime
from math import ceil

db = database('trfc.db')

app, rt = fast_app(pico=True)

In [3]:
data = {
    'min_season': 1921,
    'max_season': 2024,
    'pens_as_draw': 1,
}

venues = ('H', 'A', 'N')

min_games = 10

focus = 'm.manager_name'

In [56]:
def man_rec_query(manager = None):
    man_query = f'AND m.manager_name = "{manager}"'

    return f'''
        SELECT
            {focus},
            COUNT(*) as P,
            COUNT(
                CASE WHEN
                    (:pens_as_draw = 0 AND (c.pens_outcome IS NOT NULL AND c.pens_outcome = 'W') OR (c.pens_outcome IS NULL AND r.outcome = 'W'))
                OR 
                    (:pens_as_draw = 1 AND r.outcome = 'W')
                THEN 1 END) as W,
            COUNT(
                CASE WHEN
                    (:pens_as_draw = 0 AND (c.pens_outcome IS NOT NULL AND c.pens_outcome = 'D') OR (c.pens_outcome IS NULL AND r.outcome = 'D'))
                OR
                    (:pens_as_draw = 1 AND r.outcome = 'D')
                THEN 1 END) as D,
            COUNT(
                CASE WHEN 
                    (:pens_as_draw = 0 AND (c.pens_outcome IS NOT NULL AND c.pens_outcome = 'L') OR (c.pens_outcome IS NULL AND r.outcome = 'L'))
                OR 
                    (:pens_as_draw = 1 AND r.outcome = 'L')
                THEN 1 END) as L,
            SUM(r.goals_for) as GF,
            SUM(r.goals_against) as GA,
            SUM(r.goals_for) - SUM(r.goals_against) as GD,
            ROUND(CAST(COUNT(CASE 
                WHEN (:pens_as_draw = 0 AND (c.pens_outcome IS NOT NULL AND c.pens_outcome = 'W') OR (c.pens_outcome IS NULL AND r.outcome = 'W'))
                OR (:pens_as_draw = 1 AND r.outcome = 'W')
                THEN 1 END) AS FLOAT) / COUNT(*) * 100, 1) as win_pc
        FROM results r
        LEFT JOIN cup_game_details c ON r.game_date = c.game_date
        LEFT JOIN manager_reigns mr ON r.game_date >= mr.mgr_date_from
            AND (r.game_date <= mr.mgr_date_to OR mr.mgr_date_to IS NULL)
        LEFT JOIN managers m ON mr.manager_id = m.manager_id
        LEFT JOIN season_league_tiers slt ON r.season = slt.season AND r.game_type = 'League'
        WHERE CAST(SUBSTRING(r.season, 1, 4) AS INTEGER) >= {data['min_season']} 
            AND CAST(SUBSTRING(r.season, 1, 4) AS INTEGER) <= {data['max_season']}
            AND r.venue IN ('H', 'A', 'N')
            {f"{man_query if manager else ''}"}
        GROUP BY {focus}
        HAVING COUNT(*) >= {min_games}
        ORDER BY P DESC
    '''

In [57]:
pens_as_draw = 1

In [61]:
def man_recs(query):
    return db.execute(query, {"pens_as_draw": data['pens_as_draw']}).fetchall()

man_recs(man_rec_query('Nigel Adkins'))

[('Nigel Adkins', 48, 18, 8, 21, 67, 70, -3, 37.5)]

In [96]:
def man_rec_html(man_records):
    css = 'display: flex; flex-direction: column;'
    
    return Div(*[Div(
        *[Div(str(record[i])) for i, _ in enumerate(record)],
          id=f"{record[0]}") for record in man_records], cls=f'{css}')

In [97]:
man_rec_html(man_recs(man_rec_query('Nigel Adkins')))

```html
<div class="display: flex; flex-direction: column;">
  <div id="Nigel Adkins">
    <div>Nigel Adkins</div>
    <div>48</div>
    <div>18</div>
    <div>8</div>
    <div>21</div>
    <div>67</div>
    <div>70</div>
    <div>-3</div>
    <div>37.5</div>
  </div>
</div>

```

In [93]:
@rt('/')
def index():
    return man_rec_html(man_recs(man_rec_query('Nigel Adkins')))

In [94]:
# server = JupyUvi(app)

In [95]:
HTMX()